In [1]:
import os, glob, copy
from datetime import date
from la_forge import core as co
import numpy as np

In [47]:
basedir = '/vast/palmer/home.grace/bbl29/IPTA_DR2_analysis'
datadir = f'{basedir}/data/full_ePSRs'
template_file = './template_fullDR2_stdnoise.sh'
todays_date = str(date.today()).replace('-','')

In [48]:
# Assume 8 runs and we want 200k samples, thin of 10, assume burn of 5000 (max)
thin = 10
Nchains = 8
burn = 5000
target = 200000
single_chain_Niter = (target//Nchains+burn)*10
single_chain_Niter

300000

In [58]:
# get psrnames
psrlist = []
psrs = glob.glob(datadir + '/J*')
for psr in psrs:
    name = psr.split('/')[-1][:-5]
    psrlist.append(name)
psrlist = np.sort(np.unique(psrlist))

In [59]:
with open(template_file, "r") as f:
    template = f.read()

In [60]:
print(template)

#!/bin/bash
#SBATCH --job-name=DR2_{{PSRNAME}}_stdnoise_{{DATE}}
#SBATCH --partition={{PARTITION}}
#SBATCH --exclude=r806u23n04
#SBATCH --output=/home/bbl29/IPTA_DR2_analysis/logs/{{PSRNAME}}_DR2full_stdnoise_{{DATE}}.txt
#SBATCH --error=/home/bbl29/IPTA_DR2_analysis/logs/error/{{PSRNAME}}_DR2full_stdnoise_{{DATE}}.txt
#SBATCH --nodes=1
#SBATCH --ntasks=4
#SBATCH --cpus-per-task=1
#SBATCH --threads-per-core=1
#SBATCH --mem-per-cpu=10G
#SBATCH --time={{RUNTIME_DAYS}}-
#SBATCH --array=0-{{ARRAY_MAX}}
#SBATCH --mail-type=END
#SBATCH --mail-user=bjorn.larsen@yale.edu

echo "SLURM_JOBID: " $SLURM_JOBID
echo "SLURM_ARRAY_TASK_ID: " $SLURM_ARRAY_TASK_ID
echo "SLURM_ARRAY_JOB_ID: " $SLURM_ARRAY_JOB_ID

module load miniconda
module load OpenMPI
conda activate PTA_env

psrname={{PSRNAME}}
Niter=350000
outdir=/vast/palmer/scratch/mingarelli/bbl29/IPTA_DR2_analysis/chains/dr2full/stdnoise/{{PSRNAME}}
psr={{PSR}}
emp_dist_path=


srun -n $SLURM_NTASKS python3 ./scripts/run_std_noise.py --psrname $p

In [61]:
psrs_to_run_extra_time = ['J0613-0200','J1012+5307','J1024-0719','J1600-3053',
                          'J1643-1224','J1713+0747','J1744-1134','J1857+0943',
                          'J1909-3744','J1939+2134','J2145-0750']

In [62]:
for psrname in psrlist:
    print(psrname)
    # Write new file
    file_path = f'{basedir}/jobs/{psrname}_fullDR2_stdnoise.sh'
    job_text = copy.deepcopy(template)
    job_text = job_text.replace("{{PSRNAME}}", psrname)
    job_text = job_text.replace("{{DATE}}", todays_date)
    job_text = job_text.replace("{{PSR}}", f'{datadir}/{psrname}.hdf5')
    if psrname in psrs_to_run_extra_time:
        print('run pi_mingarelli')
        job_text = job_text.replace("{{PARTITION}}", 'pi_mingarelli')
        job_text = job_text.replace("{{RUNTIME_DAYS}}", '2')
        job_text = job_text.replace("{{ARRAY_MAX}}", '7')
    else:
        job_text = job_text.replace("{{PARTITION}}", 'scavenge')
        job_text = job_text.replace("{{RUNTIME_DAYS}}", '1')
        job_text = job_text.replace("{{ARRAY_MAX}}", '9')
    with open(file_path, "w") as f:
        f.write(job_text)

J0030+0451
J0034-0534
J0218+4232
J0437-4715
J0610-2100
J0613-0200
run pi_mingarelli
J0621+1002
J0711-6830
J0751+1807
J0900-3144
J1012+5307
run pi_mingarelli
J1022+1001
J1024-0719
run pi_mingarelli
J1045-4509
J1455-3330
J1600-3053
run pi_mingarelli
J1603-7202
J1614-2230
J1640+2224
J1643-1224
run pi_mingarelli
J1713+0747
run pi_mingarelli
J1721-2457
J1730-2304
J1732-5049
J1738+0333
J1741+1351
J1744-1134
run pi_mingarelli
J1751-2857
J1801-1417
J1802-2124
J1804-2717
J1824-2452A
J1843-1113
J1853+1303
J1857+0943
run pi_mingarelli
J1903+0327
J1909-3744
run pi_mingarelli
J1910+1256
J1911+1347
J1911-1114
J1918-0642
J1939+2134
run pi_mingarelli
J1944+0907
J1955+2908
J2010-1323
J2019+2425
J2033+1734
J2124-3358
J2129-5721
J2145-0750
run pi_mingarelli
J2229+2643
J2317+1439
J2322+2057


In [63]:
mass_submit_text = 'import os\n'
for psrname in psrlist:
    mass_submit_text += f'os.system(f"sbatch jobs/{psrname}_fullDR2_stdnoise.sh")\n'
with open(f'{basedir}/submit_DR2full_stdnoise.py', "w") as f:
    f.write(mass_submit_text)